In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

c:\users\kumar\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [4]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
print(x_train.shape,y_train.shape)

(50000, 32, 32, 3) (50000, 10)


In [6]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])        
        temp = concat
        
    return temp

In [7]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [8]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [9]:
#num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(20, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, 40, dropout_rate)
First_Transition = add_transition(First_Block, 40, dropout_rate)

Second_Block = add_denseblock(First_Transition, 40, dropout_rate)
Second_Transition = add_transition(Second_Block, 40, dropout_rate)

Third_Block = add_denseblock(Second_Transition, 40, dropout_rate)
Third_Transition = add_transition(Third_Block, 40, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  100, dropout_rate)
output = output_layer(Last_Block)


In [12]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 20)   540         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 20)   80          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 20)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [13]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [14]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 238s 5ms/step - loss: 1.5184 - acc: 0.4490 - val_loss: 1.4658 - val_acc: 0.5147

Epoch 00001: val_acc improved from -inf to 0.51470, saving model to weights.best.hdf5
Epoch 2/50
50000/50000 [==============================] - 225s 4ms/step - loss: 1.0506 - acc: 0.6247 - val_loss: 1.6784 - val_acc: 0.5020

Epoch 00002: val_acc did not improve
Epoch 3/50
50000/50000 [==============================] - 225s 4ms/step - loss: 0.8628 - acc: 0.6923 - val_loss: 1.1905 - val_acc: 0.6523

Epoch 00003: val_acc improved from 0.51470 to 0.65230, saving model to weights.best.hdf5
Epoch 4/50
50000/50000 [==============================] - 225s 5ms/step - loss: 0.7294 - acc: 0.7415 - val_loss: 1.1834 - val_acc: 0.6470

Epoch 00004: val_acc did not improve
Epoch 5/50
50000/50000 [==============================] - 225s 5ms/step - loss: 0.6550 - acc: 0.7710 - val_loss: 0.8170 - val_acc: 0.7418

Epoch 

50000/50000 [==============================] - 226s 5ms/step - loss: 0.0836 - acc: 0.9698 - val_loss: 0.6795 - val_acc: 0.8650

Epoch 00044: val_acc improved from 0.86260 to 0.86500, saving model to weights.best.hdf5
Epoch 45/50
50000/50000 [==============================] - 226s 5ms/step - loss: 0.0824 - acc: 0.9710 - val_loss: 0.7568 - val_acc: 0.8519

Epoch 00045: val_acc did not improve
Epoch 46/50
50000/50000 [==============================] - 226s 5ms/step - loss: 0.0756 - acc: 0.9730 - val_loss: 0.8820 - val_acc: 0.8307

Epoch 00046: val_acc did not improve
Epoch 47/50
50000/50000 [==============================] - 226s 5ms/step - loss: 0.0763 - acc: 0.9733 - val_loss: 0.8080 - val_acc: 0.8478

Epoch 00047: val_acc did not improve
Epoch 48/50
50000/50000 [==============================] - 226s 5ms/step - loss: 0.0782 - acc: 0.9724 - val_loss: 0.7157 - val_acc: 0.8589

Epoch 00048: val_acc did not improve
Epoch 49/50
50000/50000 [==============================] - 226s 5ms/step - 

In [17]:
model.load_weights("weights.best.hdf5")
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 17s 2ms/step
Test loss: 0.6795170318752527
Test accuracy: 0.865
